In [1]:
# pip install -r requirements.txt
# python -m ipykernel install --user --name=python3

import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dayofmonth, month, quarter, year, dayofweek, date_format, sum as spark_sum

jar_dir = "/home/bnguyen/Desktop/DE_project/scripts/jars"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    f"--jars {jar_dir}/hadoop-azure-3.3.6.jar,"
    f"{jar_dir}/azure-storage-8.6.6.jar,"
    f"{jar_dir}/hadoop-common-3.3.6.jar,"
    f"{jar_dir}/jetty-client-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-http-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-io-9.4.43.v20210629.jar,"
    f"{jar_dir}/mysql-connector-j-9.3.0.jar,"
    f"{jar_dir}/jetty-util-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-util-ajax-9.4.43.v20210629.jar "
    "pyspark-shell"
)


In [2]:
# Init spark session
spark = SparkSession.builder \
    .appName("Test CDC") \
    .getOrCreate()

25/07/16 14:04:33 WARN Utils: Your hostname, lenovo-slim resolves to a loopback address: 127.0.1.1; using 192.168.199.13 instead (on interface wlp2s0)
25/07/16 14:04:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/16 14:04:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 14:04:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/16 14:04:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/16 14:04:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# Bronze access key
spark.conf.set(
    "fs.azure.account.key.mybronze.dfs.core.windows.net",
    "c5etqTidViezB/4ukOAALy23HeMBsJJ8g+2nFaIdbC7E9PhLw0y2YIA1ItjutpqS1/8Ga8fw40mR+ASt2T+/sw=="
)

In [4]:
# Define storage account 
storage_account_bronze = "mybronze"
bronze_stream = "bronze-stream"

tables = ["Customers", "Products", 
          "Sellers", "Orders", 
          "OrderItems","ProductCategories",
          "OrderStatus","Reasons", "Payments"]


In [13]:
bronze_path = f"abfss://{bronze_stream}@{storage_account_bronze}.dfs.core.windows.net/Customers/part-*"
df = spark.read.parquet(bronze_path)
df.show()

+----------+----------+-------------------+-----------+--------------------+--------------------+---------+--------------------+--------------------+
|CustomerID|      Name|              Email|PhoneNumber|           CreatedAt|           UpdatedAt|operation|     kafka_timestamp| ingestion_timestamp|
+----------+----------+-------------------+-----------+--------------------+--------------------+---------+--------------------+--------------------+
|  10000005|Bob Wilson|bob.wilson@test.com| 0987654321|2025-07-16T07:09:35Z|2025-07-16T07:09:35Z|        c|2025-07-16 14:09:...|2025-07-16 14:09:...|
+----------+----------+-------------------+-----------+--------------------+--------------------+---------+--------------------+--------------------+

